In [1]:
! bcftools -v

bcftools 1.18
Using htslib 1.18
Copyright (C) 2023 Genome Research Ltd.
License Expat: The MIT/Expat license
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


In [1]:
# run_tag = 'baseline-kanpig'
# run_tag = 'missing-lt-pt05-mac-gte-2'
# run_tag = 'baseline-kanpig'
# run_tag = 'missing-lt-pt05-mac-gte-2-kanpig'
# run_tag = 'baseline-short-plus-sv-kanpig'
# run_tag = 'baseline-short-plus-sv-mac-gte-2-kanpig' # mac-gte-2 filter not applied yet!
run_tag = 'short-plus-kanpig-sv-hiphase-mac-gte-2-shapeit'
working_dir = f'/home/jupyter/AoU1-v1/preprocess-for-phasing/{run_tag}'
output_prefix = 'chr1_100-110Mbp'

! mkdir -p {working_dir}/phased-outputs
# ! gsutil cp 'gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/scratch/phasing/{run_tag}/*.bcf' {working_dir}/hiphase-outputs
# ! gsutil cp 'gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/scratch/phasing_kanpig/{run_tag}/*.bcf' {working_dir}/hiphase-outputs
# ! gsutil cp 'gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/scratch/phasing_kanpig/{run_tag}/*.bcf' {working_dir}/phased-outputs
! gsutil cp 'gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/submissions/b9624542-d7d2-497d-89b3-1d927a0a4302/HybridPhase/6f593bd8-a318-4ae1-8b7f-8c82269c7f21/call-Shapeit4scaffold/*.bcf' {working_dir}/phased-outputs

phased_bcf = f'{working_dir}/phased-outputs/chr1:100000000-110000000_scaffold.bcf'

phased_vcf_gz = f'{working_dir}/phased-outputs/{output_prefix}.{run_tag}.phased.vcf.gz'

! bcftools view {phased_bcf} -Oz -o {phased_vcf_gz}
! bcftools index -t {phased_vcf_gz}

Copying gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/submissions/b9624542-d7d2-497d-89b3-1d927a0a4302/HybridPhase/6f593bd8-a318-4ae1-8b7f-8c82269c7f21/call-Shapeit4scaffold/chr1:100000000-110000000_scaffold.bcf...
- [1 files][ 21.6 MiB/ 21.6 MiB]                                                
Operation completed over 1 objects/21.6 MiB.                                     


In [2]:
preprocess_dir = f'{working_dir}/pangenie-preprocess'
! mkdir -p {preprocess_dir}

In [3]:
# atomize SNP/indels

atomize_tag = 'atomize'

atomize_vcf_gz = f'{preprocess_dir}/{output_prefix}.{atomize_tag}.vcf.gz'

! bcftools norm -a {phased_vcf_gz} -Oz -o {atomize_vcf_gz}

Lines   total/split/realigned/skipped:	160575/0/0/0


In [4]:
# ! gsutil cp gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/references/T2T/trf.bb.bed {working_dir}/resources
    
# tr_bed = f'{working_dir}/resources/trf.bb.bed'

In [5]:
# tr_annot_tag = 'tr-annot'
# hiphase_concat_tr_annot_vcf_gz = f'{preprocess_dir}/{output_prefix}.{reg_af_tag}.{concat_tag}.{tr_annot_tag}.vcf.gz'
# min_overlap = 0.9

# ! bcftools view {hiphase_concat_vcf_gz} -Ou | \
#     bcftools annotate --no-version -a {tr_bed} --min-overlap :{min_overlap} -c CHROM,FROM,TO -m +TR \
#     -h <(echo '##INFO=<ID=TR,Number=0,Type=Flag,Description="Overlaps TR region with --min-overlap :0.9">') \
#     -Oz -o {hiphase_concat_tr_annot_vcf_gz}

In [6]:
# no_tr_tag = 'no-tr'
# tag = f'{reg_af_tag}.{concat_tag}.{tr_annot_tag}.{no_tr_tag}'
# hiphase_preprocessed_vcf_gz = f'{preprocess_dir}/{output_prefix}.{tag}.vcf.gz'

# ! bcftools view -i 'AF>={af_threshold} && TR=0' {hiphase_concat_tr_annot_vcf_gz} -Oz -o {hiphase_preprocessed_vcf_gz}

In [7]:
tag = f'{atomize_tag}' #.{tr_annot_tag}.{no_tr_tag}
preprocessed_vcf_gz = f'{preprocess_dir}/{output_prefix}.{tag}.vcf.gz'

In [8]:
# v3.1.0 of PanGenie run-from-callset scripts

pangenie_scripts_dir = f'/home/jupyter/AoU1-v1/pangenie/pipelines/run-from-callset/scripts'
ref_fa = f'/home/jupyter/AoU1-v1/ref/GCA_000001405.15_GRCh38_no_alt_analysis_set.fa' #chm13v2.0.ebv.fa'
run_from_callset_dir = f'{working_dir}/run-from-callset'

! mkdir -p {run_from_callset_dir}

In [9]:
# validate_vcf

! bcftools norm --check-ref e --fasta-ref {ref_fa} {preprocessed_vcf_gz} &> {run_from_callset_dir}/validate-vcf.log

In [10]:
# prepare_vcf

frac_missing = 0.2

! bcftools view {preprocessed_vcf_gz} | \
    python3 {pangenie_scripts_dir}/prepare-vcf.py --missing {frac_missing} \
    2> {run_from_callset_dir}/{output_prefix}.{tag}.prepare-vcf.log \
    1> {run_from_callset_dir}/{output_prefix}.{tag}.prepare.vcf

In [11]:
# add_ids

! cat {run_from_callset_dir}/{output_prefix}.{tag}.prepare.vcf | \
    python3 {pangenie_scripts_dir}/add-ids.py \
    2> {run_from_callset_dir}/{output_prefix}.{tag}.add-ids.log \
    1> {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.vcf

In [12]:
# normalize

! bcftools norm -m- {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.vcf \
    --threads $(nproc) \
    2> {run_from_callset_dir}/{output_prefix}.{tag}.normalize.log \
    1> {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.vcf

In [13]:
# merge_haplotypes

! python3 {pangenie_scripts_dir}/merge_vcfs.py merge \
    -vcf {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.vcf \
    -r {ref_fa} \
    -ploidy 2  \
    2> {run_from_callset_dir}/{output_prefix}.{tag}.merge-haplotypes.log \
    1> {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.mergehap.vcf

In [14]:
! bcftools view {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.mergehap.vcf \
    -Oz -o {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.mergehap.vcf.gz
! bcftools index -t {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.mergehap.vcf.gz

[W::vcf_parse] Contig 'chr1' is not defined in the header. (Quick workaround: index the file with tabix.)


In [15]:
# ! gsutil cp {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.mergehap.vcf.gz* gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/scratch/phasing/{run_tag}/
! gsutil cp {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.mergehap.vcf.gz* gs://fc-secure-f7d80b48-be60-426f-aa6b-f037a1bf7f34/scratch/phasing_kanpig/{run_tag}/

Copying file:///home/jupyter/AoU1-v1/preprocess-for-phasing/short-plus-kanpig-sv-hiphase-mac-gte-2-shapeit/run-from-callset/chr1_100-110Mbp.atomize.prepare.id.split.mergehap.vcf.gz [Content-Type=text/vcard]...
Copying file:///home/jupyter/AoU1-v1/preprocess-for-phasing/short-plus-kanpig-sv-hiphase-mac-gte-2-shapeit/run-from-callset/chr1_100-110Mbp.atomize.prepare.id.split.mergehap.vcf.gz.tbi [Content-Type=application/octet-stream]...
- [2 files][ 28.8 MiB/ 28.8 MiB]                                                
Operation completed over 2 objects/28.8 MiB.                                     


In [34]:
# # convert_back_biallelic_representation (pop bubbles and convert back to biallelic)

# ! bcftools view {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.mergehap.vcf.gz | \
#     python3 {pangenie_scripts_dir}/convert-to-biallelic-no-gq.py {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.vcf \
#     > {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.mergehap.unmergehap.vcf

In [35]:
# ! bcftools view {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.mergehap.unmergehap.vcf \
#     -Oz -o {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.mergehap.unmergehap.vcf.gz
# ! bcftools index -t {run_from_callset_dir}/{output_prefix}.{tag}.prepare.id.split.mergehap.unmergehap.vcf.gz